<a href="https://colab.research.google.com/github/joyc3lin/sqlite_database_operations/blob/main/HHA504_week3assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
#loading in packages
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
import re
import numpy as np

# **1. Data Exploration and Analysis**

In [49]:
# importing pricing transparency datasets
df_rumc = pd.read_csv('datasets/RichmondUniversityMedicalCenter_standardcharges.csv')
df_nyp = pd.read_json('datasets/NewYorkPresbyterianHospital_standardcharges.json')

In [50]:
print(df_rumc.shape)
df_rumc

(6975, 78)


,LINE TYPE,CHARGE CODE/ PACKAGE,CHARGE DESCRIPTION,DRG,CPT/HCPCS,MODIFIER,REV CODE,NDC,GROSS CHARGES,SELF PAY CASH PRICE,...,United Healthcare Community Plan IP Rate,United Healthcare Community Plan OP Rate,United Healthcare Medicaid IP Rate,United Healthcare Medicaid OP Rate,United Healthcare Medicare IP Rate,United Healthcare Medicare OP Rate,Value Options Commercial IP Rate,Value Options Commercial OP Rate,Value Options IP Rate,Value Options OP Rate
0,CDM,42126219,PELVIS HIP (PEDIATRIC)BI,NaN,73522,NaN,320.0,NaN,$581.00,$0.73,...,NaN,$0.82,NaN,$0.73,NaN,$129.01,NaN,$220.00,NaN,$220.00
1,CDM,42125039,HIP BILATERAL (OR),NaN,73522,NaN,320.0,NaN,"$1,195.00",$0.73,...,NaN,$0.82,NaN,$0.73,NaN,$129.01,NaN,$452.50,NaN,$452.50
2,CDM,42122796,"HIP,BILATERAL",NaN,73522,NaN,320.0,NaN,"$1,257.00",$0.73,...,NaN,$0.82,NaN,$0.73,NaN,$129.01,NaN,$475.50,NaN,$475.50
3,CDM,42121793,X-RAY EXAM CHEST 2 VIEWS,NaN,71046,NaN,320.0,NaN,$290.00,$1.00,...,NaN,$1.12,NaN,$1.00,NaN,$95.78,NaN,$109.50,NaN,$109.50
4,CDM,42121796,X-RAY EXAM CHEST 3 VIEWS,NaN,71047,NaN,320.0,NaN,$290.00,$1.00,...,NaN,$1.12,NaN,$1.00,NaN,$95.78,NaN,$109.50,NaN,$109.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6970,CDM,43119759,LOKELMA 10GRAM,NaN,NaN,NaN,250.0,00310-1110-01,$86.24,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,$43.12,NaN,$43.12
6971,CDM,43157874,RIVAROXABAN 15 MG TAB,NaN,NaN,NaN,250.0,50458-0578-10,$55.19,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,$27.60,NaN,$27.60
6972,CDM,36098448,BAG,NaN,NaN,NaN,270.0,NaN,$158.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,$79.00,NaN,$79.00
6973,CDM,43152941,TAMIFLU 75 MG CAPSULRE,NaN,NaN,NaN,250.0,00004-0800-85,$58.97,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,$29.49,NaN,$29.49


In [43]:
print(df_nyp.shape)
df_nyp

(6193, 47)


,Code (CPT/DRG),Description,Rev Code,Inpatient/Outpatient,Gross Charges,Discounted Cash Price,Aetna,Cigna,Empire Blue Cross Blue Shield,Emblem Health,...,Consumer Health Network,Devon,Equian,First Health,Magnacare,Multiplan/Beechstreet/PHCS,QHM,Worldwide,Minimum Negotiated Charge,Maximum Negotiated Charge
0,96360,HC IV INFUSION HYDRATION INITIAL 31 MIN-1HR,0260,Inpatient/Outpatient,866.000000,866.000000,641.706,494.486,640.84,1388.424,...,393.5104,393.5104,393.5104,393.5104,393.5104,393.5104,393.5104,393.5104,253.048250,1388.424000
1,96361,HC IV INFUSION HYDRATION FLUIDS ADDL HR,0260,Inpatient/Outpatient,358.000000,358.000000,265.278,204.418,264.92,1305.112,...,162.6752,162.6752,162.6752,162.6752,162.6752,162.6752,162.6752,162.6752,51.903250,1990.891341
2,96365,HC IV INFUSION FOR THER/PROPH/DIAG INITIAL UP ...,0260,Inpatient/Outpatient,1397.000000,1397.000000,1035.177,797.687,1033.78,1475.508,...,634.7968,634.7968,634.7968,634.7968,634.7968,634.7968,634.7968,634.7968,253.048250,1475.508000
3,96367,HC IV INF THER/PROPH/DIAG ADDL SEQ NEW DRUG UP...,0260,Inpatient/Outpatient,480.000000,480.000000,355.68,274.08,355.2,1325.12,...,218.112,218.112,218.112,218.112,218.112,218.112,218.112,218.112,82.650750,1325.120000
4,96368,HC IV INFUSION FOR THER/PROPH/DIAG CONCURRENT,0260,Inpatient/Outpatient,480.000000,480.000000,355.68,274.08,355.2,1325.12,...,218.112,218.112,218.112,218.112,218.112,218.112,218.112,218.112,156.982800,1325.120000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6188,982,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,Multiple,Inpatient,184095.129020,184095.129020,86723.5232,98911.694706,82299.84,115129.89148,...,83652.826627,83652.826627,83652.826627,83652.826627,83652.826627,83652.826627,83652.826627,83652.826627,12561.144027,115129.891480
6189,983,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,Multiple,Inpatient,85385.941857,85385.941857,57748.8352,65864.888166,63931.455093,76664.51828,...,38799.37198,38799.37198,38799.37198,38799.37198,38799.37198,38799.37198,38799.37198,38799.37198,7833.746854,76664.518280
6190,987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,Multiple,Inpatient,411665.999955,411665.999955,114736.9984,130862.199072,214294.876425,152319.20576,...,187061.03038,187061.03038,187061.03038,187061.03038,187061.03038,187061.03038,187061.03038,187061.03038,21016.798830,214294.876425
6191,988,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,Multiple,Inpatient,126522.871140,126522.871140,58654.7264,66898.093812,91907.6928,77867.13496,...,57491.992646,57491.992646,57491.992646,57491.992646,57491.992646,57491.992646,57491.992646,57491.992646,10429.428059,91907.692800


## Data Cleaning

### *Cleaning df_nyp*

In [25]:
# checking data types
for c in df_nyp.columns:
  print(f"{c}: {type(df_nyp[c][0])}")

Code (CPT/DRG): <class 'int'>
Description: <class 'str'>
Rev Code: <class 'str'>
Inpatient/Outpatient: <class 'str'>
Gross Charges: <class 'numpy.float64'>
Discounted Cash Price: <class 'numpy.float64'>
Aetna: <class 'float'>
Cigna: <class 'float'>
Empire Blue Cross Blue Shield: <class 'float'>
Emblem Health: <class 'float'>
United Health Group: <class 'float'>
Aetna Medicare: <class 'float'>
AgeWell Medicare: <class 'float'>
Emblem Medicare: <class 'float'>
Empire Medicare: <class 'float'>
Fidelis Medicare: <class 'float'>
Healthfirst Medicare: <class 'float'>
UHC Community Plan/United Medicare: <class 'float'>
VNS Medicare: <class 'float'>
WellCare Medicare: <class 'float'>
1199: <class 'int'>
Affinity Molina Essential: <class 'float'>
Affinity Molina Medicaid/CHP: <class 'float'>
Amida Care Medicaid: <class 'float'>
Emblem Medicaid/CHP: <class 'float'>
Empire Healthplus Essential: <class 'float'>
Empire Healthplus Exchange: <class 'float'>
Empire Healthplus Medicaid/CHP: <class 'flo

In [33]:
# looking for missing values
missing_values = df_nyp.isnull().sum()
missing_values

Code (CPT/DRG)                          0
Description                             0
Rev Code                                0
Inpatient/Outpatient                    0
Gross Charges                           0
Discounted Cash Price                   0
Aetna                                   0
Cigna                                   0
Empire Blue Cross Blue Shield           0
Emblem Health                           0
United Health Group                     0
Aetna Medicare                          0
AgeWell Medicare                        0
Emblem Medicare                         0
Empire Medicare                         0
Fidelis Medicare                        0
Healthfirst Medicare                    0
UHC Community Plan/United Medicare      0
VNS Medicare                            0
WellCare Medicare                       0
1199                                    0
Affinity Molina Essential               0
Affinity Molina Medicaid/CHP            0
Amida Care Medicaid               

In [24]:
# dropping row with missing data
df_nyp.dropna(inplace=True)
df_nyp.shape

(5912, 47)

In [26]:
#removing duplicate rows and columns
df_nyp.drop_duplicates(inplace=True)
df_nyp.loc[:,~df_nyp.T.duplicated(keep='first')]
print(df_nyp.shape)

(5912, 47)


In [30]:
# cleaning column names

# Function to remove white space and special characters from a value
def clean_value(value):
    cleaned_value = re.sub(r'\s+|[^a-zA-Z0-9]', '', str(value))
    return cleaned_value

df_nyp.rename(mapper=clean_value, axis=1, inplace=True)
df_nyp.sample(10)

,CodeCPTDRG,Description,RevCode,InpatientOutpatient,GrossCharges,DiscountedCashPrice,Aetna,Cigna,EmpireBlueCrossBlueShield,EmblemHealth,...,ConsumerHealthNetwork,Devon,Equian,FirstHealth,Magnacare,MultiplanBeechstreetPHCS,QHM,Worldwide,MinimumNegotiatedCharge,MaximumNegotiatedCharge
4614,D7270,HC REIMPLANTATION/STABILIZATION OF EVULSED TOOTH,0512,Inpatient/Outpatient,1080.000000,1080.000000,Not separately payable,Not separately payable,756,Not separately payable,...,490.752,490.752,490.752,490.752,490.752,490.752,490.752,490.752,211.395459,864.000000
756,81000,HC LAB URINALYSIS DIP STICK/TABLET NON-AUTO MI...,0307,Inpatient/Outpatient,97.000000,97.000000,2.814,3.5,2.814,2.814,...,44.0768,44.0768,44.0768,44.0768,44.0768,44.0768,44.0768,44.0768,2.814000,211.845065
6176,956,"LIMB REATTACHMENT, HIP AND FEMUR PROCEDURES FO...",Multiple,Inpatient,384841.352050,384841.352050,131534.0192,150019.882386,280997.545104,174618.10588,...,174871.910372,174871.910372,174871.910372,174871.910372,174871.910372,174871.910372,174871.910372,174871.910372,22857.300447,280997.545104
1150,75705,HC X-RAY ANGIOGRAPHY SPINAL SELECTIVE,0323,Inpatient/Outpatient,11556.000000,11556.000000,8562.996,4899.744,1417.0783,14437.226352,...,5251.0464,5251.0464,5251.0464,5251.0464,5251.0464,5251.0464,5251.0464,5251.0464,1009.119185,14437.226352
4789,D9430,HC DENTAL OBSERVATION VISIT,0512,Inpatient/Outpatient,290.000000,290.000000,Not separately payable,Not separately payable,203,Not separately payable,...,131.776,131.776,131.776,131.776,131.776,131.776,131.776,131.776,131.776000,396.311105
1010,72285,HC X-RAY DISCOGRAPHY CERVICAL/THORACIC SPINE,0320,Inpatient/Outpatient,6139.000000,6139.000000,4548.999,2602.936,663.6553,5393.104304,...,2789.5616,2789.5616,2789.5616,2789.5616,2789.5616,2789.5616,2789.5616,2789.5616,663.655300,5393.104304
1151,75726,HC X-RAY ANGIOGRAPHY VISCERAL SELECTIVE/SUPRAS...,0323,Inpatient/Outpatient,11563.000000,11563.000000,8568.183,4902.712,906.6277,14438.374352,...,5254.2272,5254.2272,5254.2272,5254.2272,5254.2272,5254.2272,5254.2272,5254.2272,906.627700,14438.374352
4657,D7520,HC DENTAL I&D OF EXTRAORAL ABSCESS,0512,Inpatient/Outpatient,854.000000,854.000000,Not separately payable,Not separately payable,597.8,Not separately payable,...,388.0576,388.0576,388.0576,388.0576,388.0576,388.0576,388.0576,388.0576,158.524442,683.200000
4583,D6751,HC DENTAL CROWN PORCELAIN/BASE METAL,0512,Inpatient/Outpatient,3130.000000,3130.000000,Not separately payable,Not separately payable,2191,Not separately payable,...,1422.272,1422.272,1422.272,1422.272,1422.272,1422.272,1422.272,1422.272,388.043168,2504.000000
5646,228,OTHER CARDIOTHORACIC PROCEDURES WITH MCC,Multiple,Inpatient,469589.539146,469589.539146,170400.9008,194349.136839,330231.65105,226215.86962,...,213381.486588,213381.486588,213381.486588,213381.486588,213381.486588,213381.486588,213381.486588,213381.486588,31399.757645,330231.651050


### Cleaing df_rumc

In [51]:
# checking data types
for c in df_rumc.columns:
  print(f"{c}: {type(df_rumc[c][0])}")

LINE TYPE: <class 'str'>
CHARGE CODE/ PACKAGE: <class 'str'>
CHARGE DESCRIPTION: <class 'str'>
DRG: <class 'float'>
CPT/HCPCS: <class 'str'>
MODIFIER: <class 'float'>
REV CODE: <class 'numpy.float64'>
NDC: <class 'float'>
GROSS CHARGES: <class 'str'>
SELF PAY CASH PRICE: <class 'str'>
MIN NEGOTIATED RATE: <class 'str'>
MAX NEGOTIATED RATE: <class 'str'>
Aetna Commercial IP Rate: <class 'float'>
Aetna Commercial OP Rate: <class 'str'>
Aetna Medicare IP Rate: <class 'float'>
Aetna Medicare OP Rate: <class 'str'>
Affinity Medicaid IP Rate: <class 'float'>
Affinity Medicaid OP Rate: <class 'str'>
Amerigroup Essential Plan IP Rate: <class 'float'>
Amerigroup Essential Plan OP Rate: <class 'str'>
Amerigroup Medicaid IP Rate: <class 'float'>
Amerigroup Medicaid OP Rate: <class 'str'>
Amerigroup Medicare IP Rate: <class 'float'>
Amerigroup Medicare OP Rate: <class 'str'>
Blue Cross Employee IP Rate: <class 'numpy.float64'>
Blue Cross Employee OP Rate: <class 'numpy.float64'>
Cigna Commercial I

In [52]:
# looking for missing values
missing_values2 = df_rumc.isnull().sum()
print(missing_values2)

LINE TYPE                                0
CHARGE CODE/ PACKAGE                     0
CHARGE DESCRIPTION                       0
DRG                                   5727
CPT/HCPCS                             2671
                                      ... 
United Healthcare Medicare OP Rate    3809
Value Options Commercial IP Rate      6452
Value Options Commercial OP Rate      1287
Value Options IP Rate                 6452
Value Options OP Rate                 1287
Length: 78, dtype: int64


In [56]:
#filling out cells with missing data
df_rumc = df_rumc.fillna("", inplace=False)
df_rumc.sample(10)

,LINE TYPE,CHARGE CODE/ PACKAGE,CHARGE DESCRIPTION,DRG,CPT/HCPCS,MODIFIER,REV CODE,NDC,GROSS CHARGES,SELF PAY CASH PRICE,...,United Healthcare Community Plan IP Rate,United Healthcare Community Plan OP Rate,United Healthcare Medicaid IP Rate,United Healthcare Medicaid OP Rate,United Healthcare Medicare IP Rate,United Healthcare Medicare OP Rate,Value Options Commercial IP Rate,Value Options Commercial OP Rate,Value Options IP Rate,Value Options OP Rate
1051,CDM,31290661,RPR LAC SMP FACE/EAR 12-20 CM,,12016,,450.0,,"$1,270.00",$140.00,...,,$156.80,,$140.00,,$409.44,,$263.00,,$480.50
5203,CDM,43107820,COREG 3.125MG TABLET,,,,250.0,00007-4139-20,$18.53,,...,,,,,,,,$9.26,,$9.26
3892,CDM,37681034,OT EVAL:HIGH COMPLEX TYP 60MIN,,97167,,434.0,,$491.00,,...,,,,,,$111.56,,$185.50,,$185.50
2365,DRG,APR-DRG 204-3,SYNCOPE & COLLAPSE,204-3,,,,,"$50,410.54","$7,576.85",...,"$8,486.07",,"$7,576.85",,,,,,,
2114,DRG,APR-DRG 422-1,HYPOVOLEMIA & RELATED ELECTROLYTE DISORDERS,422-1,,,,,"$15,662.56","$4,253.36",...,"$4,763.76",,"$4,253.36",,,,,,,
6392,CDM,43120105,DEXAMTHASNE 0.2 MG/ML,,J1100,,636.0,00517-4901-25,$55.23,,...,,,,,,,,$27.61,,$27.61
3957,CDM,43830257,OT NEUROMUSC RE-ED EA 30 MIN,,97112,GO,430.0,,$515.00,,...,,,,,,$40.23,,$195.00,,$195.00
5771,CDM,36036222,TRAY PEG PULL SILICONE,,,,270.0,,"$1,008.00",,...,,,,,,,,$504.00,,$504.00
5999,CDM,43106588,INVEGA TRINZA 819 MG,,J2426,,636.0,50458-0609-01,"$29,265.80",,...,,,,,,,,"$14,632.90",,"$14,632.90"
523,CDM,41220112,HOLTER MONITORING REC.,,93225,,731.0,,"$1,161.00",$18.00,...,,$20.16,,$18.00,,$132.54,,$439.50,,$439.50


In [57]:
#removing duplicate rows and columns
df_rumc.drop_duplicates(inplace=True)
df_rumc.loc[:,~df_rumc.T.duplicated(keep='first')]
print(df_rumc.shape)

(6975, 78)


In [58]:
# cleaning column names
df_rumc.rename(mapper=clean_value, axis=1, inplace=True)
df_rumc.sample(10)

,LINETYPE,CHARGECODEPACKAGE,CHARGEDESCRIPTION,DRG,CPTHCPCS,MODIFIER,REVCODE,NDC,GROSSCHARGES,SELFPAYCASHPRICE,...,UnitedHealthcareCommunityPlanIPRate,UnitedHealthcareCommunityPlanOPRate,UnitedHealthcareMedicaidIPRate,UnitedHealthcareMedicaidOPRate,UnitedHealthcareMedicareIPRate,UnitedHealthcareMedicareOPRate,ValueOptionsCommercialIPRate,ValueOptionsCommercialOPRate,ValueOptionsIPRate,ValueOptionsOPRate
1598,CPT,42808,EXCISE PHARYNX LESION,,42808,,360.0,,"$3,912.00","$2,653.05",...,,,,,,"$3,239.66",,"$1,956.00",,"$1,956.00"
353,CDM,42122242,SKULL (MIN 4 VIEWS),,70260,,320.0,,$704.00,$11.00,...,,$12.32,,$11.00,,$129.01,,$266.50,,$266.50
6455,CDM,33610668,LIDOCAINE 1%/EPI 1:200000,,,,250.0,,$2.00,,...,,,,,,,,$1.00,,$1.00
4762,DRG,MS-DRG 389,GASTROINTESTINAL OBSTRUCTION WITH CC,389,,,,,"$25,402.59",,...,,,,,"$7,870.90",,"$12,701.29",,"$12,701.29",
3673,CDM,37400003,INDIVIDUAL THERAPY 30 MIN,,90832,,900.0,,$376.00,,...,,,,,,$158.21,,$142.00,,$142.00
4027,CDM,37631152,FAMILY PSYTX W/O PT 50 MIN,,90846,,914.0,,$592.00,,...,,,,,,$158.21,,$104.00,,$104.00
6832,CDM,43199890,CATHFLO ACTIVASE IV 2MG,,J2997,,636.0,50242-0041-64,$596.93,,...,,,,,,,,$298.46,,$298.46
1605,CPT,42810,EXCISION OF NECK CYST,,42810,,360.0,,"$3,757.00","$2,653.05",...,,,,,,"$3,239.66",,"$1,878.50",,"$1,878.50"
3769,CDM,37320015,PSYCH ASSESSMNT/EVAL-EP 40 MIN,,99214,,900.0,,$427.00,,...,,,,,,,,$161.50,,$161.50
5543,CDM,43153022,EMTRIVA 200 MG CAPS,,,,250.0,54569-5521-00,$82.38,,...,,,,,,,,$41.19,,$41.19


## data distribution